In [1]:
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import time
import calendar
import datetime
api_key = 'b333a19b2c0397e8e4c1224b49b3e7cd'
api_base = 'http://ws.audioscrobbler.com/2.0/?format=json&'
user_name = 'dude0faw3'
data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key='\
                + api_key + '&user=' + user_name).json()
library = pd.read_csv('data.csv')

In [2]:
track_data = data['recenttracks']['track']
limit = data['recenttracks']['@attr']['totalPages']
page = 1
history = []
# for nested dictionary entry requests
class FakeNone:
    def get(*args):
        return FakeNone()
# ensures that there's no key error exception
def check_track(track):
    arr = [track.get('name', FakeNone()),
     track.get('artist', FakeNone()).get('#text', FakeNone()),
      track.get('date', FakeNone()).get('uts', FakeNone())
    ]
    return [None if type(x) == FakeNone else x for x in arr]

for track in track_data:
    history.append(check_track(track))
    
print('Starting! There are ' + limit + ' pages')
while int(data['recenttracks']['@attr']['page']) is not int(limit):
    if page % 25 is 0:
        print('Page %i' % page)
    page += 1
    data = http.get(api_base + 'method=user.getrecenttracks&limit=200&api_key=' + api_key + \
                    '&user=' + user_name + '&page=' + str(page)).json()
    for track in data['recenttracks']['track']:
        history.append(check_track(track))

df = pd.DataFrame(history)
df.columns = ['track_name', 'artist', 'date']
df.tail()

Starting! There are 156 pages
Page 25
Page 50
Page 75
Page 100
Page 125
Page 150


,track_name,artist,date
31229,This Is What It Feels Like,Armin van Buuren,1407180582
31230,Ecstasy,ATB,1407180320
31231,Spaceman - Drown The Fish Remix,Hardwell,1407179792
31232,Legacy,Eminem,1407179496
31233,Spaceman - Drown The Fish Remix,Hardwell,1407179059


## Goals:
- most listened to songs/ artists (as well as during a time)
- songs/artists you've neglected
- music habits (top 25 songs vs entire library, etc.)
- genres (tags)

In [3]:
# determine the n-most frequently played songs from a given pandas series
def determine_frequencies(data, num_songs, neglected=False):
    if neglected:
        counter = Counter(data)
        frequencies = counter.most_common()
        index = len(frequencies) - num_songs
        return frequencies[ index :]
    else:
        counter = Counter(data)
        return counter.most_common(num_songs)

# clean lastfm data to only have songs currently in spotify library
def refine_df(original_df, feature_name, must_haves):
    def check(val):
        for must_have in must_haves:
            if must_have == val:
                return True
        return False
    arr_check = np.vectorize(check)
    return original_df[ arr_check(original_df[ feature_name ]) ]

# return most infrequently played songs
songs = library['track_name']
df = refine_df(df, 'track_name', songs)
determine_frequencies(df['track_name'].values, 10, neglected=True)

[('Playing In The Shadows', 3),
 ('Slowly - Radio Edit', 3),
 ('Tumblr Girls', 3),
 ('Fade Away', 3),
 ('Flexicution', 2),
 ('Valerie', 2),
 ('We Come Running - Tiësto Remix', 2),
 ('Downtown Love', 2),
 ('Many Ways - Radio Edit', 2),
 ('The Birds Pt. 2 - Album Version (Edited)', 1)]

In [4]:
class TimeMachine:
    
    def __init__(self, year):
        self.seasons = {
            'spring': ( time.strptime('21 Mar ' + year, '%d %b %Y'),
                      time.strptime('21 Jun ' + year, '%d %b %Y')),
            'summer': ( time.strptime('21 Jun ' + year, '%d %b %Y'),
                      time.strptime('23 Sep ' + year, '%d %b %Y')),
            'fall': ( time.strptime('23 Sep ' + year, '%d %b %Y'),
                      time.strptime('21 Dec ' + year, '%d %b %Y')),
            'winter': ( time.strptime('21 Dec ' + year, '%d %b %Y'),
                        time.strptime('21 Mar ' + str(int(year) + 1), '%d %b %Y'))
        }
        self.year = year
        
    def set_year(self, new_year):
        self = self.__init__(new_year)
        
    def in_season(self, season, df):
        timeslice = self.seasons[season]
        def check_time(seconds):
            listen_time =  time.gmtime(int(seconds))
            return listen_time > timeslice[0] and listen_time < timeslice[1]
        arr_time_check = np.vectorize(check_time)
        return df[arr_time_check(df['date'])]
    
    def in_year(self, df):
        date = datetime.date(int(self.year), 1, 1)
        def check_time(seconds):
            listen_time =  datetime.date.fromtimestamp(int(seconds))
            return listen_time.year == date.year
        arr_time_check = np.vectorize(check_time)
        return df[ arr_time_check(df['date'])]
    
# make a timemachine to 2014, check out top artists from then
obj = TimeMachine('2014')
summer_df = obj.in_year(df)
determine_frequencies(summer_df['artist'], 20)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'